In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
!pip install tensorly
import tensorly as tl
from tensorly.decomposition import tucker
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error
from tensorly.tenalg import multi_mode_dot
from tensorly import tensor as tl_tensor
import time
from tensorly import tucker_to_tensor

# Data loading and check for NaNs (land data) that need to be masked

In [ ]:
# Import datasets
train_tensor = np.load("/content/drive/MyDrive/LSE Capstone G Drive/Train Val Test Sets/Tensor 4D/train_tensor_new_split.npy")
val_tensor = np.load("/content/drive/MyDrive/LSE Capstone G Drive/Train Val Test Sets/Tensor 4D/val_tensor_new_split.npy")
test_tensor = np.load("/content/drive/MyDrive/LSE Capstone G Drive/Train Val Test Sets/Tensor 4D/test_tensor_new_split.npy")

print(f"Train tensor shape: {train_tensor.shape}")
print(f"Val tensor shape: {val_tensor.shape}")
print(f"Test tensor shape: {test_tensor.shape}")

In [ ]:
nan_count = np.isnan(train_tensor).sum()
T, V, H, W = train_tensor.shape
print(f"Number of NaNs in train_tensor: {nan_count}")
print(f"Train tensor values: {T*V*H*W}")

nan_count = np.isnan(val_tensor).sum()
T, V, H, W = val_tensor.shape
print(f"Number of NaNs in val_tensor: {nan_count}")
print(f"Val tensor values: {T*V*H*W}")

nan_count = np.isnan(test_tensor).sum()
T, V, H, W = test_tensor.shape
print(f"Number of NaNs in test_tensor: {nan_count}")
print(f"test tensor values: {T*V*H*W}")

Number of NaNs in train_tensor: 11172120
Train tensor values: 51704856
Number of NaNs in val_tensor: 3541920
Val tensor values: 16392096
Number of NaNs in test_tensor: 5388240
test tensor values: 24936912


# Tucker Decomposition

## Tucker decomposition function: used only on the training set

In [ ]:
def decompose_spatial_modes(tensor, r_lat, r_lon, r_time=None):
    T, V, Lat, Lon = tensor.shape

    # Check if val_tensor has any NaNs before proceeding
    if np.isnan(tensor).any():
        print("⚠️ Warning: NaNs detected in val_tensor. Proceeding to fill with 0.0.")
        tensor = np.nan_to_num(tensor, nan=0.0)
    else:
        tensor = tensor.copy()
        print("Train has no NaNs.")

    tensor_tl = tl_tensor(tensor)

    print(f"Running Tucker on tensor with shape: {tensor.shape}")
    start = time.time()

    # Use full rank for time if not specified
    if r_time is None:
        r_time = T

    # Set ranks for all modes: time, vars, lat, lon
    ranks = [r_time, V, r_lat, r_lon]

    core, factors = tucker(
        tensor_tl,
        rank=ranks,
        n_iter_max=100,
        tol=1e-5
    )

    print(f"Decomposition complete in {time.time() - start:.2f}s")

    print("Type of factors:", type(factors))
    print("Length of factors:", len(factors))
    print("Types of each factor:", [type(f) for f in factors])
    print("Shapes (if available):", [getattr(f, 'shape', 'N/A') for f in factors])

    U_time, U_vars, U_lat, U_lon = factors

    # Convert to NumPy arrays if needed
    U_time = np.array(U_time)
    U_vars = np.array(U_vars)
    U_lat = np.array(U_lat)
    U_lon = np.array(U_lon)

    print(f"U_time shape: {U_time.shape}")
    print(f"U_vars shape: {U_vars.shape}")
    print(f"U_lat shape: {U_lat.shape}")
    print(f"U_lon shape: {U_lon.shape}")

    core_tensor = core
    print("Core tensor shape:", core_tensor.shape)

    return core_tensor, U_time, U_vars, U_lat, U_lon

## Compute one-step ahead VAR forecasts and evaluate the metrics

In [ ]:
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_pred) + np.abs(y_true)) + 1e-8
    return np.mean(numerator / denominator) * 100


def one_step_var_forecast(core_sequence, var_model):
    """
    Forecast next timestep using previous (VAR(1)) from a sequence of core vectors.
    core_sequence: [T+1, D] including last train step + rest of val/test
    Returns: forecasted [T, D]
    """
    T, D = core_sequence.shape
    forecasts = []
    for t in range(1, T):
        y_pred = var_model.forecast(core_sequence[t-1:t], steps=1)[0]
        forecasts.append(y_pred)
    return np.array(forecasts)


def evaluate_split(original_tensor, reconstructed_tensor, split_name, r_time, r_lat, r_lon):
    """
    Computes per-variable RMSE, MAE, SMAPE for a given data split
    """
    mask = ~np.isnan(original_tensor)
    metrics = []
    V = original_tensor.shape[1]

    for v in range(V):
        y_true = original_tensor[:, v, :, :][mask[:, v, :, :]]
        y_pred = reconstructed_tensor[:, v, :, :][mask[:, v, :, :]]

        rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))
        mae = np.mean(np.abs(y_pred - y_true))
        smape_val = smape(y_true, y_pred)

        metrics.append({
            'split': split_name,
            'variable': v,
            'r_time': r_time,
            'r_lat': r_lat,
            'r_lon': r_lon,
            'RMSE': rmse,
            'MAE': mae,
            'SMAPE': smape_val
        })

    return metrics

# Rank approximation for the tensor
$$
X = \sum_{r=1}^R a_r^{(1)} \times \cdots \times a_r^{(N)}
$$
The rank of a tensor denoted by $rank(X)$ is the smallest positive integer $R$ such that the above decomposition is exact, where ”exact” means that the equality
holds above. A decomposition of a tensor in the form where $R$ = $rank(X)$ is called the rank decomposition of $X$.

## Function for Tucker Decomposition
In terms of factor analysis, the Tucker Decomposition has a multi-linear factor form, consisting of a low-dimensional tensor that is assumed to vary over time and a set of fixed factor loading matrices that do not vary over time. Therefore, the Tucker Decomposition relates the high-dimensional tensor observations to low-dimensional tensor factors.

$M_t$ and $E_t$ are the corresponding signal and noise components of $X_t$, respectively. We assume that $E_t$ are uncorrelated across time. Tucker decomposition approximates the signal \(M_t\) as follows:

$$
    X_t = {M}_t + {E}_t = {F}_t \times_1 U_1 \times_2 U_2 + {E}_t
$$

${F}_t \in \mathbb{R}^{r_1 \times r_2}$ is the time-dependent core tensor (compressed representation) capturing latent ocean dynamics, which we would like to model over time. $U_k$ are the {factor loading matrices}, fixed over time: $U_1 \in \mathbb{R}^{d_1 \times r_1}$ is the spatial factor matrix and $U_2 \in \mathbb{R}^{d_2 \times r_2}$ is the ocean charactersitics factor matrix.


In [ ]:
def evaluate_tucker_ranks(r_time_vals, lat_lon_pairs):

    for r_time in r_time_vals:
        for r_lat, r_lon in lat_lon_pairs:
            print(f"\nRunning Tucker with ranks - Time: {r_time}, Lat: {r_lat}, Lon: {r_lon}")

            # Reload the train and validation sets just to be safe
            train_tensor = np.load("/content/drive/MyDrive/LSE Capstone G Drive/Tensor Creation/train_tensor.npy")
            val_tensor = np.load("/content/drive/MyDrive/LSE Capstone G Drive/Tensor Creation/val_tensor.npy")
            test_tensor = np.load("/content/drive/MyDrive/LSE Capstone G Drive/Tensor Creation/test_tensor.npy")

            core_tensor, U_time, U_vars, U_lat, U_lon = decompose_spatial_modes(train_tensor, r_lat, r_lon, r_time)

            # Flatten for VAR
            F_matrix = core_tensor.reshape(core_tensor.shape[0], -1)
            print(f"F_matrix for VAR shape: {F_matrix.shape}")

            # Run VAR
            model = VAR(F_matrix.T)
            results = model.fit(maxlags=1)
            print(results.summary())

            # --- Evaluate TRAIN ---
            reconstructed_train = tucker_to_tensor((core_tensor, [U_time, U_vars, U_lat, U_lon]))
            if np.isnan(train_tensor).any():
                print("⚠️ Warning: NaNs detected in val_tensor. Using mask to ignore them during evaluation.")
                mask_train = ~np.isnan(train_tensor)
            else:
                mask_train = np.ones_like(train_tensor, dtype=bool)
                print("Train has no NaNs.")

            for v in range(train_tensor.shape[1]):
                original_v = train_tensor[:, v, :, :]
                recon_v = reconstructed_train[:, v, :, :]
                mask_v = mask_train[:, v, :, :]

                y_true = original_v[mask_v]
                y_pred = recon_v[mask_v]

                mse_v = np.mean((y_pred - y_true) ** 2)
                rmse_v = np.sqrt(mse_v)
                mae_v = np.mean(np.abs(y_pred - y_true))
                smape_v = smape(y_true, y_pred)

                records.append({
                    'split': 'train',
                    'variable': v,
                    'r_time': r_time,
                    'r_lat': r_lat,
                    'r_lon': r_lon,
                    'RMSE': rmse_v,
                    'MAE': mae_v,
                    'SMAPE': smape_v
                })

            # --- Evaluate VALIDATION ---
            # Check if val_tensor has any NaNs before proceeding
            if np.isnan(val_tensor).any():
                print("⚠️ Warning: NaNs detected in val_tensor. Proceeding to fill with 0.0.")
                val_tensor_filled = np.nan_to_num(val_tensor, nan=0.0)
                used_nan_filling = True
            else:
                val_tensor_filled = val_tensor.copy()
                used_nan_filling = False

            # Project to core space
            core_val = multi_mode_dot(
                tl.tensor(val_tensor_filled),
                [U_vars.T, U_lat.T, U_lon.T],
                modes=[1, 2, 3]
            )

            # Reconstruct
            reconstructed_val = multi_mode_dot(
                core_val,
                [U_vars, U_lat, U_lon],
                modes=[1, 2, 3]
            )

            # Mask for evaluation
            if np.isnan(val_tensor).any():
                print("⚠️ Warning: NaNs detected in val_tensor. Using mask to ignore them during evaluation.")
                mask_val = ~np.isnan(val_tensor)
                used_masking = True
            else:
                mask_val = np.ones_like(val_tensor, dtype=bool)
                used_masking = False

            # Optional: summary print
            if not used_nan_filling and not used_masking:
                print("✅ No NaNs detected in val_tensor. No filling or masking applied.")

            for v in range(val_tensor.shape[1]):
                original_v = val_tensor[:, v, :, :]
                recon_v = reconstructed_val[:, v, :, :]
                mask_v = mask_val[:, v, :, :]

                y_true = original_v[mask_v]
                y_pred = recon_v[mask_v]

                mse_v = np.mean((y_pred - y_true) ** 2)
                rmse_v = np.sqrt(mse_v)
                mae_v = np.mean(np.abs(y_pred - y_true))
                smape_v = smape(y_true, y_pred)

                records.append({
                    'split': 'validation',
                    'variable': v,
                    'r_time': r_time,
                    'r_lat': r_lat,
                    'r_lon': r_lon,
                    'RMSE': rmse_v,
                    'MAE': mae_v,
                    'SMAPE': smape_v
                })

    # Save to DataFrame and CSV
    df_results = pd.DataFrame(records)
    df_results.to_csv("tucker_eval_per_variable.csv", index=False)
    print("Saved all results to 'tucker_eval_per_variable.csv'")

# Evaluating this model for various ranks
We show that for certain pairs, this model fails to work; such as when time_rank = 100, variable_rank = 8, lat_rank = 2, lon_rank = 2. While the tucker decomposition works, the VAR model tries to fit a matrix with 100 time points and 32 features -> (100,32) and fails because there are too few observations to estimate the 32x32 coefficient matrix.

In [ ]:
# r_time_vals = [20, 50, 100, 150]
# lat_lon_pairs = [(5, 10), (15, 20), (25, 30), (35, 40), (50, 60), (60, 80), (60, 100), (60, 150)]
r_time_vals = [400]
lat_lon_pairs = [(2, 2)]

# Run the rank approximation
records = []
evaluate_tucker_ranks(r_time_vals, lat_lon_pairs)

# Save to CSV
df_metrics = pd.DataFrame(records)
df_metrics.to_csv("tucker_metrics_results_safe.csv", index=False)
print("Saved detailed results to tucker_metrics_results.csv")


Running Tucker with ranks - Time: 100, Lat: 2, Lon: 2
Train has no NaNs.
Running Tucker on tensor with shape: (593, 8, 63, 173)
Decomposition complete in 117.90s
Type of factors: <class 'list'>
Length of factors: 4
Types of each factor: [<class 'numpy.ndarray'>, <class 'numpy.ndarray'>, <class 'numpy.ndarray'>, <class 'numpy.ndarray'>]
Shapes (if available): [(593, 100), (8, 8), (63, 2), (173, 2)]
U_time shape: (593, 100)
U_vars shape: (8, 8)
U_lat shape: (63, 2)
U_lon shape: (173, 2)
Core tensor shape: (100, 8, 2, 2)
F_matrix for VAR shape: (100, 32)


LinAlgError: 33-th leading minor of the array is not positive definite